<a href="https://colab.research.google.com/github/jfrxelza3im/Twitter_Analysis_Sentiment/blob/main/twitter_analysis_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
! mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
  0% 0.00/80.9M [00:00<?, ?B/s]
100% 80.9M/80.9M [00:00<00:00, 1.37GB/s]


In [ ]:
from zipfile import ZipFile
dataset = "/content/sentiment140.zip"
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('done')


done


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

In [ ]:
TWITTER_DATA=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [ ]:
 TWITTER_DATA.shape

(1599999, 6)

In [ ]:
 TWITTER_DATA.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:

column_names=['target','ids','date','flag','user','text']
TWITTER_DATA=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1',names=column_names)

In [ ]:
 TWITTER_DATA.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
 TWITTER_DATA.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
 TWITTER_DATA['target'].value_counts()

,count
target,
0,800000
4,800000


In [ ]:
  TWITTER_DATA.replace({'target':{4:1}},inplace=True)

In [ ]:
TWITTER_DATA['target'].value_counts()

,count
target,
0,800000
1,800000


0-> negative tweet
1-> postive tweet

**STEMMING**

STEMMING IS THE PROCESS OF REDUCIG WORD TO ITS ROOT WORD

In [ ]:
#on ne peut pas utiliser directement PorterStemmer, car c’est une classe.
#Il faut créer une instance (un objet) de cette classe pour pouvoir appeler sa méthode .stem().
port_stem=PorterStemmer()

### **re.sub**: remplacer une partie d'une chaîne de caractères qui correspond à un motif (pattern) par une autre chaîne.

#re.sub(pattern, repl, string, count=0, flags=0)

pattern : l'expression régulière à rechercher.
 repl : la chaîne de remplacement (ou une fonction).
 string : la chaîne de caractères sur laquelle on applique la substitution.
 count (optionnel) : le nombre maximum de remplacements (0 = illimité).
 flags (optionnel) : pour des options comme re.IGNORECASE, etc

text = "Mon numéro est 123-456-7890"

masked = re.sub(r"\d", "X", text)
print(masked)  # Mon numéro est XXX-XXX-XXXX

In [ ]:
def stemming(content):
     stemmed_content=re.sub('[^a-zA-Z]',' ',content)
     stemmed_content=stemmed_content.lower()
     stemmed_content=stemmed_content.split()
     stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
     stemmed_content=' '.join(stemmed_content) #convert list to string
     return stemmed_content

In [ ]:
#CREAT NEW COLUMN OF THE TEXT AFTER NETTOIYAGE
TWITTER_DATA['stemmed_content']=TWITTER_DATA['text'].apply(stemming)

In [ ]:
 #the length of text( NUMBER OF CHARS)
TWITTER_DATA['text_length']=TWITTER_DATA['stemmed_content'].apply(len)

In [ ]:
#Number of words
TWITTER_DATA['word_count']=TWITTER_DATA['stemmed_content'].apply(lambda x:len(x.split()))


In [ ]:
#If a user tweets several times in a row, their subsequent tweets will follow the tone of the first.
TWITTER_DATA.sort_values(['user','date'],inplace=True)

TWITTER_DATA['date']=pd.to_datetime(TWITTER_DATA['date'])
# Écart de temps entre deux tweets consécutifs du même user
TWITTER_DATA['diff_time']=TWITTER_DATA.groupby('user')['date'].diff().dt.total_seconds().fillna(999999)
# Nouveau groupe si plus de 5 minutes d'écart
TWITTER_DATA['burst_id']=TWITTER_DATA.groupby('user')['diff_time'].transform(lambda x:(x>300).cumsum())
# (Optionnel) marquer le premier tweet de chaque "rafale"
TWITTER_DATA['is_first_twitte']=TWITTER_DATA.groupby(['user','burst_id']).cumcount()==0
# Créer une nouvelle feature : "emotion du début de la rafale"
TWITTER_DATA['burst_target']=TWITTER_DATA.groupby(['user','burst_id'])['target'].transform('first')
#.transform('first')  reply the first vlaue of target for each burst_id group
TWITTER_DATA['same_burst_as_first_emotion']=(TWITTER_DATA['target']==TWITTER_DATA['burst_target']).astype(int)


In [ ]:
TWITTER_DATA.head()

,target,ids,date,flag,user,text,stemmed_content,text_length,diff_time,burst_id,is_first_twitte,burst_target,same_burst_as_first_emotion,word_count
849111,1,1565014948,2009-04-20 05:39:01,NO_QUERY,000catnap000,@Wumbologist huh uhu...i see dead people xD ab...,wumbologist huh uhu see dead peopl xd abhalt k...,62,999999.0,1,True,1,1,12
1042668,1,1957229112,2009-05-28 23:53:20,NO_QUERY,000catnap000,awake since six o'clock... i had two hours sle...,awak sinc six clock two hour sleep anyway go v...,75,3348859.0,2,True,1,1,13
1177063,1,1981357110,2009-05-31 08:59:07,NO_QUERY,000catnap000,this is one of the cartoons i used to watch to...,one cartoon use watch togeth brother http bit ...,67,205547.0,3,True,1,1,13
1422100,1,2058432528,2009-06-06 14:55:54,NO_QUERY,000catnap000,@Wumbologist hmmm...why not,wumbologist hmmm,16,539807.0,4,True,1,1,2
1425085,1,2058981540,2009-06-06 16:00:48,NO_QUERY,000catnap000,can't get enough of it http://bit.ly/134sSZ,get enough http bit ly ssz,26,3894.0,5,True,1,1,6


In [ ]:
 print(TWITTER_DATA[['target','stemmed_content']])


         target                                    stemmed_content
849111        1  wumbologist huh uhu see dead peopl xd abhalt k...
1042668       1  awak sinc six clock two hour sleep anyway go v...
1177063       1  one cartoon use watch togeth brother http bit ...
1422100       1                                   wumbologist hmmm
1425085       1                         get enough http bit ly ssz
...         ...                                                ...
1168901       1  wat mild surpriseeeeeeeee mah dear bro c min b...
1391604       1                                      lovee mmi xxx
1466556       1  tanoshii tanoshiiiiiiiiiiiiiiii thak god dat e...
329490        0    omg im workin clinic tonight last client cancel
764016        0               wish anywher margheritavil come mind

[1600000 rows x 2 columns]


In [ ]:
X_full=TWITTER_DATA[['stemmed_content','same_burst_as_first_emotion','text_length','word_count']]
Y=TWITTER_DATA['target']

 *splitting the data to train and test data (x_train,y_train)80% (x_test,y_test)20%*







In [ ]:
 X_train_df,X_test_df,Y_train,Y_test=train_test_split(X_full,Y,test_size=0.2,stratify=Y,random_state=2)


In [ ]:
print(X_full.shape,X_train_df.shape,X_test_df.shape)

(1600000, 4) (1280000, 4) (320000, 4)


In [ ]:
print(X_train_df)

                                           stemmed_content  \
904322                                     go chantal aunt   
1212555  hoptonhousebnb probabl still get paid load rec...   
79319                  usair lame unaccompani minor polici   
417659                                   someon bring food   
540404              mani cooki stomach waaaaaaah sad panda   
...                                                    ...   
1383291  chadmichmurray catcher rye favorit book high s...   
1302211                               iammaxathotspot want   
1542510                       found true blood onlin yesss   
887310   alifanat hi ye kinda like movi well im drink c...   
177564                                              acho q   

         same_burst_as_first_emotion  text_length  word_count  
904322                             1           15           3  
1212555                            1           78          13  
79319                              1           35           5  

In [ ]:
print(X_test_df)

*'X_train,X_test,Y_train,Y_test sont des liste' *

In [ ]:

vectorizer=TfidfVectorizer()
 X_text_train=vectorizer.fit_transform(X_train_df['stemmed_content'])
 X_text_test=vectorizer.transform(X_test_df['stemmed_content'])

info: Tokenisation : coupe le texte en mots (ou tokens)

In [ ]:
# Features numériques supplémentaires
features = ['same_burst_as_first_emotion', 'text_length', 'word_count']
X_extra_train=X_train_df[features].to_numpy()
X_extra_test=X_test_df[features].to_numpy()

In [ ]:
print(X_extra_train)

[[ 1 15  3]
 [ 1 78 13]
 [ 1 35  5]
 ...
 [ 1 28  5]
 [ 1 68 14]
 [ 1  6  2]]


In [ ]:
print(X_extra_test )

In [ ]:
 from scipy.sparse import csr_matrix
X_extra_train_sparse=csr_matrix(X_extra_train)
X_extra_test_sparse=csr_matrix(X_extra_test)

In [ ]:
 from scipy.sparse import hstack
X_train_final=hstack([X_text_train,X_extra_train_sparse])
X_test_final=hstack([X_text_test,X_extra_test_sparse])

In [ ]:
 model=LogisticRegression(max_iter=1000, verbose=1)

In [ ]:
model.fit(X_train_final ,Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.7min finished


LogisticRegression(max_iter=1000, verbose=1)

In [ ]:
X_train_preddiction=model.predict(X_train_final)
training_accuracy=accuracy_score(X_train_preddiction,Y_train)
print('the accuracy score on the training data',training_accuracy)

the accuracy score on the training data 0.78627734375


In [ ]:
# accuracy of test data
X_test_preddiction=model.predict(X_test_final)
test_accuracy=accuracy_score(X_test_preddiction,Y_test)
print('the accuracy score on the test data',test_accuracy)

the accuracy score on the test data 0.77409375


In [ ]:
import pickle

In [ ]:
#saving the model avoid re-train each time
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

using the saved model for future predictions

In [ ]:
#loading the saved model
loaded_model=pickle.load(open('trained_model.sav','rb'))

In [ ]:
X_new=X_test[10]
print(Y_test[10])
prediction=loaded_model.predict(X_new)
print(prediction)

NameError: name 'X_test' is not defined